In [165]:
base_url = './assignment1-data/'
root_en = 'training.en'
root_de = 'training.de'
root_es = 'training.es'

In [166]:
import re
import sys
import numpy as np
import pandas as pd
from random import random
from math import log
from collections import defaultdict


tri_counts=defaultdict(int) #counts of all trigrams in input


#this function currently does nothing.
def preprocess_line(line):
    line = re.sub(r'\d', '0', line)  # Convert digits to '0'
    rule = re.compile(r'[^a-z0 .]')  # Define allowed characters
    line = rule.sub('', line.lower())  # Remove disallowed characters and lowercase
    return line


#This bit of code gives an example of how you might extract trigram counts
#from a file, line by line. If you plan to use or modify this code,
#please ensure you understand what it is actually doing, especially at the
#beginning and end of each line. Depending on how you write the rest of
#your program, you may need to modify this code.

en_tri_counts = defaultdict(int)
de_tri_counts = defaultdict(int)
es_tri_counts = defaultdict(int)

enfile = base_url + root_en
defile = base_url + root_de
esfile = base_url + root_es

enset = set([])
deset = set([])
esset = set([])

def process_file(file_path, trigram_counts, vocabulary):
    with open(file_path) as f:
        for line in f:
            line = preprocess_line(line)
            for ch in line:
                vocabulary.add(ch)
            for j in range(len(line) - 2):
                trigram = line[j:j + 3]
                trigram_counts[trigram] += 1

process_file(enfile, en_tri_counts, enset)
process_file(defile, de_tri_counts, deset)
process_file(esfile, es_tri_counts, esset)



print(len(en_tri_counts))
print(len(de_tri_counts))
print(len(es_tri_counts))

print(len(enset))

#Some example code that prints out the counts. For small input files
#the counts are easy to look at but for larger files you can redirect
#to an output file (see Lab 1).
# print("Trigram counts in ", infile, ", sorted alphabetically:")
# for trigram in sorted(tri_counts.keys()):
#     print(trigram, ": ", tri_counts[trigram])
print("Trigram counts in ", defile, ", sorted numerically:")
for tri_count in sorted(de_tri_counts.items(), key=lambda x:x[1], reverse = True):
    print(tri_count[0], ": ", str(tri_count[1]))

3452
3943
3068
29
Trigram counts in  ./assignment1-data/training.de , sorted numerically:
en  :  3704
er  :  1927
ich :  1671
 de :  1648
der :  1438
die :  1311
ie  :  1287
 di :  1221
ch  :  1067
ung :  1046
 un :  988
n d :  985
sch :  978
 be :  966
ein :  912
nd  :  876
den :  875
che :  823
cht :  808
 da :  762
und :  737
ng  :  715
gen :  700
ten :  660
in  :  653
 au :  652
eit :  627
 ei :  591
ion :  577
 in :  556
hen :  553
 zu :  544
es  :  537
on  :  536
 wi :  518
ine :  506
 ge :  505
ent :  490
ber :  489
nen :  487
nde :  470
 vo :  469
wir :  459
 fr :  453
lic :  449
ver :  442
 si :  436
n u :  435
men :  432
te  :  428
 ko :  426
n w :  419
ter :  415
ere :  407
t d :  402
ht  :  402
nge :  390
it  :  390
r d :  387
en. :  384
n s :  384
 ve :  379
 we :  374
e d :  370
rde :  370
her :  364
run :  355
ste :  344
000 :  344
ese :  340
n a :  331
ies :  330
auf :  325
n e :  325
 pr :  318
 er :  316
ren :  316
zu  :  310
n b :  307
 mi :  300
as  :  300
mit :  29

In [167]:
file_path = base_url + 'model-br.en'

with open(file_path, 'r') as file:
    lines = file.readlines()

lines[:20]

['   \t3.333e-02\n',
 '  #\t3.333e-02\n',
 '  .\t3.333e-02\n',
 '  0\t3.333e-02\n',
 '  a\t3.333e-02\n',
 '  b\t3.333e-02\n',
 '  c\t3.333e-02\n',
 '  d\t3.333e-02\n',
 '  e\t3.333e-02\n',
 '  f\t3.333e-02\n',
 '  g\t3.333e-02\n',
 '  h\t3.333e-02\n',
 '  i\t3.333e-02\n',
 '  j\t3.333e-02\n',
 '  k\t3.333e-02\n',
 '  l\t3.333e-02\n',
 '  m\t3.333e-02\n',
 '  n\t3.333e-02\n',
 '  o\t3.333e-02\n',
 '  p\t3.333e-02\n']

In [168]:
trigrams = []
probabilities = []

pretrained_set = defaultdict(float)

for line in lines:
    trigram, prob = line.split('\t')
    trigrams.append(trigram)
    prob_float = float(prob.strip())
    pretrained_set[trigram] = prob_float
    probabilities.append(float(prob))

probabilities_np = np.array(probabilities)

probability_stats = {
    'min_probability': np.min(probabilities_np),
    'max_probability': np.max(probabilities_np),
    'mean_probability': np.mean(probabilities_np),
    'median_probability': np.median(probabilities_np),
    'std_deviation': np.std(probabilities_np),
    'total_unique_trigrams': len(trigrams)
}

zero_probabilities_count = np.sum(probabilities_np == 0)

probability_stats['zero_probabilities_count'] = zero_probabilities_count

probability_stats_df = pd.DataFrame([probability_stats])

print(probability_stats_df)

   min_probability  max_probability  mean_probability  median_probability  \
0          0.00001           0.9986           0.03337             0.03333   

   std_deviation  total_unique_trigrams  zero_probabilities_count  
0       0.078584                  26100                         0  


In [169]:
probability_bins = {
    'very_small_probabilities': np.sum((probabilities_np > 0) & (probabilities_np <= 0.001)),
    'small_probabilities': np.sum((probabilities_np > 0.001) & (probabilities_np <= 0.01)),
    'moderate_probabilities': np.sum((probabilities_np > 0.01) & (probabilities_np <= 0.1)),
    'large_probabilities': np.sum(probabilities_np > 0.1),
}

very_small_prob_trigrams = [trigrams[i] for i in range(len(probabilities_np)) if probabilities_np[i] <= 0.001]

unique_very_small_prob_trigrams_count = len(set(very_small_prob_trigrams))

smoothing_analysis = {
    'total_trigrams': len(trigrams),
    'unique_trigrams_with_very_small_probabilities': unique_very_small_prob_trigrams_count,
    'distribution_of_probabilities': probability_bins
}

smoothing_analysis

{'total_trigrams': 26100,
 'unique_trigrams_with_very_small_probabilities': 5097,
 'distribution_of_probabilities': {'very_small_probabilities': np.int64(5097),
  'small_probabilities': np.int64(4004),
  'moderate_probabilities': np.int64(16029),
  'large_probabilities': np.int64(970)}}

In [170]:
def build_trigram_model(file_path):
    trigram_counts = defaultdict(int)
    bigram_counts = defaultdict(int)
    
    with open(file_path, 'r') as file:
        for line in file:
            line = preprocess_line(line)
            for i in range(len(line) - 2):
                bigram = line[i:i+2] 
                trigram = line[i:i+3]
                bigram_counts[bigram] += 1
                trigram_counts[trigram] += 1

    trigram_probabilities = defaultdict(float)
    for trigram in trigram_counts:
        bigram = trigram[:2]
        trigram_probabilities[trigram] = trigram_counts[trigram] / bigram_counts[bigram]

    return trigram_probabilities
    

In [171]:
def save_trigram_model(trigram_probabilities, output_file):
    with open(output_file, 'w') as f:
        for trigram, prob in sorted(trigram_probabilities.items()):
            f.write(f"{trigram}\t{prob:.6e}\n")
        

# Trigram without smoothing (overfitted)

In [172]:
trigram_probabilities_en = build_trigram_model(enfile)

output_file = 'trigram_model.en'
save_trigram_model(trigram_probabilities_en, output_file)

print("Trigram counts in ", defile, ", sorted numerically:")
for tri_count in sorted(trigram_probabilities_en.items(), key=lambda x:x[1], reverse = True):
    print(tri_count[0], ": ", str(tri_count[1]))

Trigram counts in  ./assignment1-data/training.de , sorted numerically:
py  :  1.0
ium :  1.0
dfu :  1.0
equ :  1.0
bje :  1.0
kum :  1.0
yth :  1.0
 qu :  1.0
sda :  1.0
kno :  1.0
i.e :  1.0
.e. :  1.0
e.  :  1.0
lre :  1.0
by  :  1.0
lwa :  1.0
cqu :  1.0
dte :  1.0
ijv :  1.0
jva :  1.0
aes :  1.0
dne :  1.0
dhe :  1.0
sgr :  1.0
ez  :  1.0
nzl :  1.0
zle :  1.0
bc  :  1.0
ajo :  1.0
tme :  1.0
dge :  1.0
yin :  1.0
mly :  1.0
eje :  1.0
hdr :  1.0
 .. :  1.0
... :  1.0
a00 :  1.0
c00 :  1.0
lco :  1.0
0ec :  1.0
oje :  1.0
bmi :  1.0
lgi :  1.0
gy  :  1.0
pme :  1.0
p.m :  1.0
.m. :  1.0
m00 :  1.0
uum :  1.0
vy  :  1.0
aor :  1.0
yme :  1.0
lki :  1.0
tne :  1.0
fh  :  1.0
uk  :  1.0
nqu :  1.0
iho :  1.0
xod :  1.0
lmo :  1.0
0.0 :  1.0
0st :  1.0
rha :  1.0
fma :  1.0
wth :  1.0
fwa :  1.0
tde :  1.0
mfu :  1.0
pgr :  1.0
mcc :  1.0
tgi :  1.0
azo :  1.0
kwa :  1.0
dbo :  1.0
pne :  1.0
hwh :  1.0
npo :  1.0
jim :  1.0
uxe :  1.0
nwi :  1.0
wpo :  1.0
yce :  1.0
nbu :  1.0
fsu 

# Trigram probabilities models

In [183]:
trigram_probabilities_de = build_trigram_model(defile)
trigram_probabilities_es = build_trigram_model(esfile)

output_file_de = 'trigram_model.de'
save_trigram_model(trigram_probabilities_de, output_file_de)

output_file_es = 'trigram_model.es'
save_trigram_model(trigram_probabilities_es, output_file_es)

# Trigram with Add-1 Smoothing

In [174]:
def build_trigram_model_with_smoothing(file_path, alpha=1):
    trigram_counts = defaultdict(int)
    bigram_counts = defaultdict(int)
    
    with open(file_path, 'r') as file:
        for line in file:
            line = preprocess_line(line)
            for i in range(len(line) - 2):
                bigram = line[i:i+2]  # other 2 characters seen in whole corpus
                trigram = line[i:i+3]  # three characters seen
                bigram_counts[bigram] += 1
                trigram_counts[trigram] += 1

    trigram_probabilities = defaultdict(float)
    vocabulary_size = len(enset) 

    for trigram in trigram_counts:
        bigram = trigram[:2]
        trigram_probabilities[trigram] = (trigram_counts[trigram] + alpha) / (bigram_counts[bigram] + alpha * vocabulary_size)

    return trigram_probabilities

In [175]:
trigram_probabilities_smoothing = build_trigram_model_with_smoothing(enfile)

output_file = 'trigram_model_smoothing.en'
save_trigram_model(trigram_probabilities_smoothing, output_file)

print("Trigram counts in ", defile, ", sorted numerically:")
for tri_count in sorted(trigram_probabilities_smoothing.items(), key=lambda x:x[1], reverse = True):
    print(tri_count[0], ": ", str(tri_count[1]))

Trigram counts in  ./assignment1-data/training.de , sorted numerically:
of  :  0.9349670122525919
ion :  0.8835666912306559
for :  0.8834586466165414
ld  :  0.8610169491525423
ly  :  0.8594059405940594
by  :  0.825
you :  0.8229166666666666
to  :  0.8140531276778064
ive :  0.7954545454545454
ts  :  0.7894736842105263
ng  :  0.7894073139974779
eur :  0.7815126050420168
ds  :  0.7735849056603774
ed  :  0.7694090382387022
eve :  0.7681159420289855
ave :  0.7665369649805448
ike :  0.7642857142857142
equ :  0.7565217391304347
 re :  0.7543689320388349
ry  :  0.7536945812807881
jec :  0.75
he  :  0.7159288950853956
ty  :  0.7125984251968503
oin :  0.712
mbe :  0.7105263157894737
pme :  0.7083333333333334
ake :  0.7058823529411765
ide :  0.696
yme :  0.6956521739130435
par :  0.6940509915014165
nd  :  0.6932610379550735
bje :  0.6923076923076923
 qu :  0.6923076923076923
ey  :  0.6896551724137931
my  :  0.6851851851851852
 th :  0.6824634199606902
mr  :  0.6746411483253588
000 :  0.6679104477

In [176]:
pretrained_set

defaultdict(float,
            {'   ': 0.03333,
             '  #': 0.03333,
             '  .': 0.03333,
             '  0': 0.03333,
             '  a': 0.03333,
             '  b': 0.03333,
             '  c': 0.03333,
             '  d': 0.03333,
             '  e': 0.03333,
             '  f': 0.03333,
             '  g': 0.03333,
             '  h': 0.03333,
             '  i': 0.03333,
             '  j': 0.03333,
             '  k': 0.03333,
             '  l': 0.03333,
             '  m': 0.03333,
             '  n': 0.03333,
             '  o': 0.03333,
             '  p': 0.03333,
             '  q': 0.03333,
             '  r': 0.03333,
             '  s': 0.03333,
             '  t': 0.03333,
             '  u': 0.03333,
             '  v': 0.03333,
             '  w': 0.03333,
             '  x': 0.03333,
             '  y': 0.03333,
             '  z': 0.03333,
             ' . ': 0.03333,
             ' .#': 0.03333,
             ' ..': 0.03333,
             ' .0': 0.03

In [177]:
# Filter and print trigrams containing '#'
for trigram, prob in trigram_probabilities_smoothing.items():
    if 'a' in trigram:
        print(trigram, ": ", prob)


cla :  0.11392405063291139
lar :  0.21363636363636362
are :  0.2655954631379962
pea :  0.24673202614379086
ean :  0.19094766619519093
an  :  0.17301038062283736
 pa :  0.17723342939481268
par :  0.6940509915014165
arl :  0.13137996219281664
lia :  0.13191489361702127
iam :  0.25595238095238093
ame :  0.43373493975903615
t a :  0.09586964759378552
 ad :  0.03433777154870357
adj :  0.008875739644970414
ida :  0.034666666666666665
day :  0.24701195219123506
ay  :  0.5739130434782609
0 a :  0.10582010582010581
 an :  0.3114926419060967
and :  0.47923875432525953
e a :  0.0964504738896116
 ag :  0.03258584442887176
aga :  0.14736842105263157
gai :  0.21212121212121213
ain :  0.46153846153846156
u a :  0.09937888198757763
 a  :  0.14085494043447794
a h :  0.028112449799196786
 ha :  0.5962099125364432
hap :  0.015625
app :  0.5435897435897435
yea :  0.411214953271028
ear :  0.173974540311174
ar  :  0.08979206049149338
tha :  0.12506486766995328
hat :  0.4476102941176471
at  :  0.376598465473

In [178]:
def evaluate_perplexity(trigram_probabilities, validation_file):
    log_prob_sum = 0
    trigram_count = 0

    with open(validation_file, 'r') as file:
        for line in file:
            line = preprocess_line(line)
            for i in range(len(line) - 2):
                trigram = line[i:i+3]
                trigram_count += 1

                probability = trigram_probabilities.get(trigram, 1e-10)
                log_prob_sum += log(probability, 2)

    avg_log_prob = log_prob_sum / trigram_count
    perplexity = 2 ** (-avg_log_prob)
    
    return perplexity

# Generation of the text

In [179]:
import random

def generateFromLM(pretrained_set, initial_sequence='##', length=300):
    sequence = initial_sequence
    while len(sequence) < length:
        history = sequence[-2:]
        possible_next_chars = {}
        
        for trigram, prob in pretrained_set.items():
            if trigram.startswith(history):
                # print('trigram', trigram)
                next_char = trigram[2]
                # print('next_char', next_char)
                possible_next_chars[next_char] = prob
                # print('possible_next_chars', possible_next_chars)
        if not possible_next_chars:
            # Fallback: restart with a common initial sequence or choose a random trigram
            sequence += random.choice(list(pretrained_set.keys()))[2]
        else:
            # Choose the next character based on the probabilities
            next_char = random.choices(list(possible_next_chars.keys()), weights=possible_next_chars.values(), k=1)[0]
            sequence += next_char

    return sequence

# Example usage
generated_sequence = generateFromLM(pretrained_set)
print(generated_sequence)


##man teres.#leah.#ress.#jud wit but mys.#are.#i his.#what therettere.#el for.#0xquilkis.#can the hou boy done melike the this your the he tod.#noseve hats take me.##yount#res.#ilke does.#flown gonny dows.#cand he you play the mak its there.#an a lock.#xl me.#candy.#lookaboy lettl.#st make san duck 


In [180]:
generated_sequence = generateFromLM(trigram_probabilities_en, initial_sequence='  ')
print(generated_sequence)

  parned fority hentrans.ammity fif enspe. 00000000000 as the sesivend menaliketion clent prit int con of th i tareagive itere ithas me ine par whaveld bes fical wort.itedinis greseeks reary refly for majore whiph we se a s the commis eurang thicauseland to posionce progragaing con eu a proculd vere


In [186]:
generated_sequence_de = generateFromLM(trigram_probabilities_de, initial_sequence='  ')
print(generated_sequence_de)

In [ ]:
generated_sequence_es = generateFromLM(trigram_probabilities_es, initial_sequence='  ')
print(generated_sequence_es)

In [185]:
file_path = base_url + 'test'

perplexity_pretrained = evaluate_perplexity(pretrained_set, file_path)
print(perplexity_pretrained)

perplexity = evaluate_perplexity(trigram_probabilities_en, file_path)
print(perplexity)

perplexity_de = evaluate_perplexity(trigram_probabilities_de, file_path)
print(perplexity_de)

perplexity_es = evaluate_perplexity(trigram_probabilities_es, file_path)
print(perplexity_es)



22.666517240777555
15.612854574082895
491.07879350138063
1141.239457929304


In [182]:
# generated_sequence = random.choice(list(trigram_probabilities.keys()))[:2]